In [44]:
import pandas as pd
import glob

In [45]:
# files = ["shr/shr_2015/shr_2015.xlsx", "shr/shr_2014/shr_2014.xlsx", "shr/shr_2013/shr_2013.xlsx", "shr/shr_2012/shr_2012.xlsx", "shr/shr_2011/shr_2011.xlsx", "shr/shr_2010/shr_2010.xlsx", "shr/shr_2009/shr_2009.xlsx", "shr/shr_2008/shr_2008.xlsx", "shr/shr_2007/shr_2007.xlsx", "shr/shr_2006/shr_2006.xlsx"]

# all_data = pd.DataFrame()
# for f in files:
#     print(f)
#     df = pd.read_excel(f)
#     all_data = all_data.append(df,ignore_index=True)





df = pd.read_excel("shr/shr_2015/shr_2015.xlsx")

In [46]:
list(all_data.columns)

['Additional Offender Count',
 'Additional Victim Count',
 'Agency Name',
 'County',
 'Geographic Division',
 'Group',
 'Identifier Code',
 'Incident Number',
 'Last Update',
 'MSA Code',
 'MSA Indication',
 'Month of Offense',
 'Numeric State Code',
 'ORI Code',
 'Offender 1 : Weapon',
 'Offender 10: Age',
 'Offender 10: Circumstance',
 'Offender 10: Ethnic Origin',
 'Offender 10: Race',
 'Offender 10: Relationship to First Victim',
 'Offender 10: Sex',
 'Offender 10: Sub-Circumstance',
 'Offender 10: Weapon',
 'Offender 11: Age',
 'Offender 11: Circumstance',
 'Offender 11: Ethnic Origin',
 'Offender 11: Race',
 'Offender 11: Relationship to First Victim',
 'Offender 11: Sex',
 'Offender 11: Sub-Circumstance',
 'Offender 11: Weapon',
 'Offender 1: Age',
 'Offender 1: Circumstance',
 'Offender 1: Ethnic Origin',
 'Offender 1: Race',
 'Offender 1: Relationship to First Victim',
 'Offender 1: Sex',
 'Offender 1: Sub-circumstance',
 'Offender 2: Age',
 'Offender 2: Circumstance',
 'Offen

In [47]:
death_eligible_circumstances = { 3, 5, 2, 9, 46, 49, 48 }
circumstance_columns = ["V29","V79","V87","V95","V103","V111","V119","V127","V135","V143","V151"]
age_columns = ["V19", "V33", "V37", "V41", "V45", "V49", "V53", "V57", "V61", "V65", "V69"]
offender_age_columns = ["V23","V73","V81","V89","V97","V105","V113","V121","V129","V137","V145"]


def label_unique(row):
    return '' + row['V3'] + '-' + str(row['V17'])

def label_death_eligible(row):    
    num_16 = count_under_16(row)
    
    if not check_offender_age(row): #stops if all perps are less than 16
        return 0
    
    if row['V31'] > 0 or check_age(row): #multiple victims or very young victim
        return 1 + row['V32'] - num_16
    return check_circumstance(row) #based on circumstance of each perp

def check_circumstance(row):
    count = 0
    for column in circumstance_columns:
        if (circumstance_help(row[column])):
            count += 1
    return count

def circumstance_help(val):
    return val in death_eligible_circumstances

def check_age(row):
    for column in age_columns:
        if (age_help(row[column])):
            return True
    return False

def age_help(val):
    return val != ' ' and (val == 'BB' or val == 'NB' or (val < 6 and val != 0))

def check_offender_age(row):
    for off_age in offender_age_columns:
        if offender_age_help(row[off_age]):
            return True;
    return False;
    
def offender_age_help(val):
    return val != ' ' and type(val) == int and (val == 0 or val >= 16)

def count_under_16(row):
    count = 0
    for col in offender_age_columns:
        val = row[col]
        if val != ' ' and type(val) == int and (val != 0 and val < 16):
            count += 1
    return count

In [48]:
df['death_eligible_count'] = df.apply(label_death_eligible, axis=1)

In [51]:
df['death_eligible_count'].value_counts()

0    11868
1     1482
2      304
3      103
4       59
5       14
6        8
8        4
7        2
Name: death_eligible_count, dtype: int64

In [50]:
df.to_csv("shr_2015_death_eligible.csv")